In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [ ]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/tag/fhvhv/fhvhv_tripdata_2021-01.csv.gz

In [3]:
!wc -l fhvhv_tripdata_2021-01.csv

11908469 fhvhv_tripdata_2021-01.csv


In [4]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [5]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [6]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [9]:
import pandas as pd

In [10]:
df_pandas = pd.read_csv('head.csv')

In [11]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [12]:
from pyspark.sql import types

In [13]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [14]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [15]:
df = df.repartition(24)

In [ ]:
df.write.parquet('fhvhv/2021/01/')

In [17]:
df = spark.read.parquet('fhvhv/2021/01/')

In [18]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [19]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003') \
  .show(5)

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-02 23:41:01|2021-01-02 23:44:14|          20|          20|
|2021-01-01 17:38:33|2021-01-01 17:42:16|         254|         254|
|2021-01-01 02:31:50|2021-01-01 02:42:34|         173|         260|
|2021-01-01 22:59:30|2021-01-01 23:13:00|          49|         226|
|2021-01-01 09:34:40|2021-01-01 09:38:49|         212|         213|
+-------------------+-------------------+------------+------------+
only showing top 5 rows



SELECT * FROM df WHERE hvfhs_license_num =  HV0003

In [20]:
from pyspark.sql import functions as F

In [21]:
df \
    .withColumn('pickup_date',F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date',F.to_date(df.dropoff_datetime)) \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show(5)

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-01|  2021-01-01|         163|          48|
| 2021-01-01|  2021-01-01|         117|         201|
| 2021-01-02|  2021-01-02|          20|          20|
| 2021-01-01|  2021-01-01|         254|         254|
| 2021-01-01|  2021-01-01|         173|         260|
+-----------+------------+------------+------------+
only showing top 5 rows



In [22]:
df.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2021-01-01 14:07:50|2021-01-01 14:15:19|         163|          48|   null|
|           HV0005|              B02510|2021-01-01 22:00:44|2021-01-01 22:09:32|         117|         201|   null|
|           HV0003|              B02835|2021-01-02 23:41:01|2021-01-02 23:44:14|          20|          20|   null|
|           HV0003|              B02882|2021-01-01 17:38:33|2021-01-01 17:42:16|         254|         254|   null|
|           HV0003|              B02764|2021-01-01 02:31:50|2021-01-01 02:42:34|         173|         260|   null|
+-----------------+--------------------+-------------------+-------------------+

In [23]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}' #hexadecimal
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [24]:
crazy_stuff('B02884')

's/b44'

In [25]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [27]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show(5)

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/9ce| 2021-01-01|  2021-01-01|         163|          48|
|  e/9ce| 2021-01-01|  2021-01-01|         117|         201|
|  s/b13| 2021-01-02|  2021-01-02|          20|          20|
|  e/b42| 2021-01-01|  2021-01-01|         254|         254|
|  e/acc| 2021-01-01|  2021-01-01|         173|         260|
+-------+-----------+------------+------------+------------+
only showing top 5 rows



In [28]:
!head -n 10 head.csv

hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
HV0003,B02682,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,
HV0003,B02682,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,
HV0003,B02764,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,
HV0003,B02764,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,
HV0003,B02764,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,
HV0005,B02510,2021-01-01 00:06:59,2021-01-01 00:43:01,88,42,
HV0005,B02510,2021-01-01 00:50:00,2021-01-01 01:04:57,42,151,
HV0003,B02764,2021-01-01 00:14:30,2021-01-01 00:50:27,71,226,
HV0003,B02875,2021-01-01 00:22:54,2021-01-01 00:30:20,112,255,
